# Data Envelopment Analysis

In [55]:
import pandas as pd
from pulp import *

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [56]:
df_input = pd.read_csv("data/inputs.csv", sep=';')
input_columns = list(df_input.columns[1:])
df_input.columns = ['name'] + input_columns

df_output = pd.read_csv("data/outputs.csv", sep=';')
output_columns = list(df_output.columns[1:])
df_output.columns = ['name'] + output_columns

df = pd.merge(df_input, df_output, on='name')
df

,name,i1,i2,i3,i4,o1,o2
0,WAW,10.500,36,129.400,7.000,9.500,129.700
1,KRK,3.100,19,31.600,7.900,2.900,31.300
2,KAT,3.600,32,57.400,10.500,2.400,21.100
3,WRO,1.500,12,18.000,3.000,1.500,18.800
4,POZ,1.500,10,24.000,4.000,1.300,16.200
5,LCJ,0.600,12,24.000,3.900,0.300,4.200
6,GDN,1.000,15,42.900,2.500,2.000,23.600
7,SZZ,0.700,10,25.700,1.900,0.300,4.200
8,BZG,0.300,6,3.400,1.200,0.300,6.200
9,RZE,0.600,6,11.300,2.700,0.300,3.500


## Obliczanie efektywności

In [59]:
def efficiency():
    solutions = {}
    for idx_dmu, dmu in df.iterrows():
        
        problem = LpProblem("dea", LpMinimize)
        
        theta = LpVariable("theta", 0)
        
        decision_variables = {}
        for idx in df.index:
            decision_variables[idx] = LpVariable(f"x_{idx}", 0)
    
        for column in df.columns:
            if column in input_columns:
                problem += lpSum(value*decision_variables[idx] for value, idx in zip(df.loc[:, column], df.index)) <= dmu[column]*theta
            if column in output_columns:
                problem += lpSum(value*decision_variables[idx] for value, idx in zip(df.loc[:, column], df.index)) >= dmu[column]
    
        problem += theta
        problem.solve(solver=GLPK(msg=False))
        solution = {variable.name: variable.varValue for variable in problem.variables()}
        solutions[idx_dmu] = solution

    return {idx: solution for idx, solution in solutions.items()}

In [60]:
results = efficiency()
for idx, sol in results.items():
    # print(f"DMU index: {idx}, DMU name: {df.loc[idx, 'name']}, Efficiency: {eff:.3f}")
    print(f"{df.loc[idx, 'name']} & {sol['theta']:.3f} \\\\")

WAW & 1.000 \\
KRK & 1.000 \\
KAT & 0.591 \\
WRO & 1.000 \\
POZ & 0.800 \\
LCJ & 0.300 \\
GDN & 1.000 \\
SZZ & 0.271 \\
BZG & 1.000 \\
RZE & 0.409 \\
IEG & 0.258 \\


# Obliczanie HCU i poprawek

In [62]:
for idx, sol in results.items():

    if sol['theta'] < 1:  
        print(df.loc[idx, 'name'], end="")
        for input_column in input_columns:
            hcu = sum(value*_lambda for value, _lambda in zip(df.loc[:, input_column], [v for k, v in sorted([(k, v) for k, v in sol.items() if k.startswith('x')], key=lambda x: int(x[0].split('_')[1]))]))
            print(f" & {hcu:.3f}", end="")
        for input_column in input_columns:
            hcu = sum(value*_lambda for value, _lambda in zip(df.loc[:, input_column], [v for k, v in sorted([(k, v) for k, v in sol.items() if k.startswith('x')], key=lambda x: int(x[0].split('_')[1]))]))
            improvement = df.loc[idx, input_column] - hcu
            print(f" & {improvement:.3f}", end="")    
        print(" \\\\")

KAT & 2.128 & 18.919 & 33.935 & 4.396 & 1.472 & 13.081 & 23.465 & 6.104 \\
POZ & 1.200 & 7.998 & 19.195 & 1.928 & 0.300 & 2.002 & 4.805 & 2.072 \\
LCJ & 0.180 & 2.783 & 7.201 & 0.475 & 0.420 & 9.217 & 16.799 & 3.425 \\
SZZ & 0.190 & 2.708 & 6.959 & 0.465 & 0.510 & 7.292 & 18.741 & 1.435 \\
RZE & 0.246 & 2.455 & 4.624 & 0.537 & 0.354 & 3.545 & 6.676 & 2.163 \\
IEG & 0.026 & 0.388 & 1.109 & 0.065 & 0.074 & 9.612 & 62.291 & 2.935 \\


## Obliczanie superefektywności

In [63]:
solutions = {}
for idx_dmu in df.index:
    
    problem = LpProblem("dea", LpMaximize)
    
    decision_variables_v = {column: LpVariable(f"v_{column}", 0) for column in input_columns}
    decision_variables_u = {column: LpVariable(f"u_{column}", 0) for column in output_columns}
    
    problem += lpSum(value*variable for value, variable in zip(df.loc[idx_dmu, input_columns], decision_variables_v.values())) == 1

    for idx, dmu2 in df.iterrows():
        if idx == idx_dmu:
            continue
        problem += lpSum(value*variable for value, variable in zip(df.loc[idx, output_columns], decision_variables_u.values())) <= lpSum(value*variable for value, variable in zip(df.loc[idx, input_columns], decision_variables_v.values()))

    problem += lpSum(value*variable for value, variable in zip(df.loc[idx_dmu, output_columns], decision_variables_u.values()))
    problem.solve(solver=GLPK(msg=False))
    solution = {variable.name: variable.varValue for variable in problem.variables()}
    solutions[idx_dmu] = solution

In [64]:
for idx, solution in solutions.items():
    super_efficiency = sum(value*variable for value, variable in zip(df.loc[idx, output_columns], [v for k, v in solution.items() if k.startswith('u')]))
    # print(f"DMU index: {idx}, DMU name: {df.loc[idx, 'name']}, Efficiency: {super_efficiency:.3f}")
    print(f"{df.loc[idx, 'name']} & {super_efficiency:.3f} \\\\")

WAW & 2.278 \\
KRK & 1.124 \\
KAT & 0.591 \\
WRO & 1.040 \\
POZ & 0.800 \\
LCJ & 0.300 \\
GDN & 2.000 \\
SZZ & 0.271 \\
BZG & 1.746 \\
RZE & 0.409 \\
IEG & 0.258 \\


# Obliczanie efektywności krzyżowej

In [69]:
df_cross = pd.DataFrame(0.0, index=df.index, columns=df.index)
sol = efficiency()
for idx_dmu in df.index:
    decision_variables_v = {column: LpVariable(f"v_{column}", 0) for column in input_columns}
    decision_variables_u = {column: LpVariable(f"u_{column}", 0) for column in output_columns}

    sums_inputs = {column: df.drop(index=idx_dmu)[column].sum() for column in input_columns}
    sums_outputs = {column: df.drop(index=idx_dmu)[column].sum() for column in output_columns}

    problem = LpProblem("dea", LpMinimize)

    problem += lpSum(value*variable for value, variable in zip(sums_inputs.values(), decision_variables_v.values())) == 1

    for idx, dmu2 in df.iterrows():
        if idx == idx_dmu:
            problem += lpSum(value*variable for value, variable in zip(df.loc[idx, output_columns], decision_variables_u.values())) == sol[idx_dmu]['theta']*lpSum(value*variable for value, variable in zip(df.loc[idx, input_columns], decision_variables_v.values()))
        else:
            problem += lpSum(value*variable for value, variable in zip(df.loc[idx, output_columns], decision_variables_u.values())) <= lpSum(value*variable for value, variable in zip(df.loc[idx, input_columns], decision_variables_v.values()))

    problem += lpSum(value*variable for value, variable in zip(sums_outputs.values(), decision_variables_u.values()))
    problem.solve(solver=GLPK(msg=False))
    solution = {variable.name: variable.varValue for variable in problem.variables()}

    for idx_dmu2 in df.index:
        if idx_dmu == idx_dmu2:
            df_cross.loc[idx_dmu, idx_dmu2] = sol[idx_dmu]['theta']
        else:
            numerator = sum(value*variable for value, variable in zip(df.loc[idx_dmu2, output_columns], [v for k, v in solution.items() if k.startswith('u')]))
            denominator = sum(value*variable for value, variable in zip(df.loc[idx_dmu2, input_columns], [v for k, v in solution.items() if k.startswith('v')]))
            df_cross.loc[idx_dmu, idx_dmu2] = round(numerator/denominator, 3)
    
df_cross

,0,1,2,3,4,5,6,7,8,9,10
0,1.000,0.214,0.108,0.338,0.219,0.058,0.509,0.119,0.279,0.070,0.011
1,0.800,1.000,0.456,0.908,0.590,0.136,0.508,0.127,0.961,0.289,0.001
2,0.913,1.000,0.591,1.000,0.774,0.259,1.000,0.238,0.973,0.409,0.002
3,0.997,1.000,0.470,1.000,0.649,0.153,0.615,0.156,1.000,0.295,0.003
4,1.000,1.000,0.563,1.000,0.800,0.255,1.000,0.243,0.909,0.403,0.006
5,0.595,0.491,0.278,0.605,0.512,0.300,1.000,0.261,1.000,0.273,0.078
6,0.452,0.468,0.333,0.500,0.433,0.250,1.000,0.214,0.500,0.250,0.025
7,1.000,0.755,0.371,0.856,0.737,0.273,1.000,0.271,1.000,0.346,0.036
8,0.550,0.543,0.202,0.573,0.370,0.096,0.302,0.090,1.000,0.170,0.005
9,0.903,0.996,0.591,1.000,0.770,0.261,1.000,0.238,1.000,0.409,0.002


In [70]:
for idx in df_cross.index:
    print(df.loc[idx, 'name'], end="")
    for idx2 in df_cross.index:
        print(f" & {df_cross.loc[idx, idx2]:.3f}", end="")
    print(f" & {df_cross.mean()[idx]:.3f}", end="")
    print(" \\\\")

WAW & 1.000 & 0.214 & 0.108 & 0.338 & 0.219 & 0.058 & 0.509 & 0.119 & 0.279 & 0.070 & 0.011 & 0.794 \\
KRK & 0.800 & 1.000 & 0.456 & 0.908 & 0.590 & 0.136 & 0.508 & 0.127 & 0.961 & 0.289 & 0.001 & 0.718 \\
KAT & 0.913 & 1.000 & 0.591 & 1.000 & 0.774 & 0.259 & 1.000 & 0.238 & 0.973 & 0.409 & 0.002 & 0.383 \\
WRO & 0.997 & 1.000 & 0.470 & 1.000 & 0.649 & 0.153 & 0.615 & 0.156 & 1.000 & 0.295 & 0.003 & 0.756 \\
POZ & 1.000 & 1.000 & 0.563 & 1.000 & 0.800 & 0.255 & 1.000 & 0.243 & 0.909 & 0.403 & 0.006 & 0.574 \\
LCJ & 0.595 & 0.491 & 0.278 & 0.605 & 0.512 & 0.300 & 1.000 & 0.261 & 1.000 & 0.273 & 0.078 & 0.213 \\
GDN & 0.452 & 0.468 & 0.333 & 0.500 & 0.433 & 0.250 & 1.000 & 0.214 & 0.500 & 0.250 & 0.025 & 0.812 \\
SZZ & 1.000 & 0.755 & 0.371 & 0.856 & 0.737 & 0.273 & 1.000 & 0.271 & 1.000 & 0.346 & 0.036 & 0.201 \\
BZG & 0.550 & 0.543 & 0.202 & 0.573 & 0.370 & 0.096 & 0.302 & 0.090 & 1.000 & 0.170 & 0.005 & 0.863 \\
RZE & 0.903 & 0.996 & 0.591 & 1.000 & 0.770 & 0.261 & 1.000 & 0.238 & 1.0

In [71]:
df_cross.mean()

0    0.794
1    0.718
2    0.383
3    0.756
4    0.574
5    0.213
6    0.812
7    0.201
8    0.863
9    0.287
10   0.039
dtype: float64